This is my first attempt at creating a classifier.

First section: Importing libraries and defining global variables

In [21]:
import argparse
import logging
import torch
import torch.nn as nn
from torch.utils import data, model_zoo
import numpy as np
import pickle
from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys
import os
import os.path as ospbce
import matplotlib.pyplot as plt
import random
from tensorboardX import SummaryWriter
from PIL import Image
import cv2
import sys
sys.path.append('/home/danleh/graduation-project-2020-sonnefred/multi-head/')

from model.discriminator import FCDiscriminator
from util.Loss import MSE_loss, CE_Loss, CrossEntropy2d
from model.res_deeplab_dn import ResNet_Deeplab
from util.dataset_dn import Freiburg_Dataset
from util import transform
from util.util import AverageMeter, intersectionAndUnion

BATCH_SIZE = 16
NUM_WORKERS = 4
IGNORE_LABEL = 255
INPUT_SIZE = '640,320'
LEARNING_RATE = 2.5e-4
LEARNING_RATE_D = 1e-5
MOMENTUM = 0.9
ALPHA = 0.9
NUM_CLASSES = 14
START_EPOCH = 0
FIRST_EPOCH = 50
EPOCHS = 150
POWER = 0.9
RANDOM_SEED = 1234
RESTORE_FROM = '/data/models/multi-head/multitask_v2/trained_weights.pth'
ROOT = '/data/freiburg/'
#LOG_SAVE_PATH = '/home/jsun/Project/multi-head/logs/dn_v3/'
#SNAPSHOT_DIR = '/home/jsun/Project/multi-head/snapshots/dn_v3/'
#BEST_DIR = '/home/jsun/Project/multi-head/best/dn_v3/'
WEIGHT_DECAY = 0.0005

GPU = 0

PATH_DNV3 = '/data/models/multi-head/dn_v3/trained_weights.pth'
np.random.seed(0)
torch.manual_seed(0)
IM_WIDTH=640
IM_HEIGHT=320

Modified Dataset which contains the same input images as dn_v3, but where the labels refer to the domain of the images. The dataset now only needs to be called per split (training/testing) & no longer separately for each domain

In [22]:
def make_dataset(split=None, data_root=None):
    data_list_day = os.path.join(data_root, 'train', 'day', 'file_list.txt')
    data_list_night = os.path.join(data_root, 'train', 'night', 'file_list.txt')

    if not os.path.isfile(data_list_day):
        raise (RuntimeError("Image list file do not exist: " + data_list_day + "\n"))
    if not os.path.isfile(data_list_night):
        raise (RuntimeError("Image list file do not exist: " + data_list_night + "\n"))

    image_label_list = []
    
    list_read_day = open(data_list_day).readlines()
    list_read_night = open(data_list_night).readlines()
    
    
    length80day=(0.8*list_read_day.__len__()).__round__()
    length80night=(0.8*list_read_night.__len__()).__round__()

    if split=='train':
        start_day,start_night=0,0
        end_day,end_night=length80day,length80night
    else:
        start_day,start_night=length80day+1,length80night+1
        end_day,end_night=list_read_day.__len__(),list_read_night.__len__()
    
    for line in list_read_day[start_day:end_day]:
        line = line.strip()
        image_name_day = os.path.join(data_root, 'train', 'day', 'RGB', 'fl_rgb'+line+'.png').replace('\\', '/')
        label_name_day= 'day'
        item_day = (image_name_day, label_name_day)
        image_label_list.append(item_day)
    for line in list_read_night[start_night:end_night]:
        line = line.strip()
        image_name_night = os.path.join(data_root, 'train', 'night', 'RGB', 'fl_rgb'+line+'.png').replace('\\', '/')
        label_name_night = 'night'
        item_night = (image_name_night, label_name_night)
        image_label_list.append(item_night)

            
    print("Checking image&label pair {} list done!".format(split))
    return image_label_list


class Classifier_Dataset(data.Dataset):
    def __init__(self, split, data_root=None, transform=None):
        self.split = split
        self.data_list = make_dataset(split, data_root)
        self.transform = transform
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        image_path, label = self.data_list[index]
        
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.float32(img) / 255.0
        img = cv2.resize(img,(640,320),interpolation=cv2.INTER_AREA)

        if self.transform is not None:
            img, nothing = self.transform(img, img[:,:,0])
        
        return img, label

In [23]:
train_transform = transform.Compose([
    transform.RandomHorizontalFlip(),
    transform.ToTensor(),
    transform.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])

test_transform = transform.Compose([
    transform.ToTensor(),
    transform.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])

# create dataloader   
train_data= Classifier_Dataset('train', ROOT, train_transform)
train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
#train_night_loader = data.DataLoader(Freiburg_Dataset('train', ROOT, 'night', train_transform), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

test_data = Classifier_Dataset('test', ROOT, train_transform)
test_loader = data.DataLoader(test_data, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
#test_data_night = Freiburg_Dataset('test', ROOT, 'night', test_transform)
#test_loader_night = data.DataLoader(test_data_night, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

Checking image&label pair train list done!
Checking image&label pair test list done!


In [24]:
class dnClassifier(nn.Module):
    def __init__(self, img_width, img_height):
        super(dnClassifier, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=10, stride = 10)
        self.maxpool1=nn.MaxPool2d(kernel_size=15)
        self.fc1 = nn.Linear(40, 1)
        
    # Prediction
    def forward(self, x):
        x = self.cnn1(x)
        x = self.maxpool1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [25]:
img_width=IM_WIDTH
img_height=IM_HEIGHT
model=dnClassifier(img_width, img_height)
device = torch.device("cuda")
model.to(device)

dnClassifier(
  (cnn1): Conv2d(3, 5, kernel_size=(10, 10), stride=(10, 10))
  (maxpool1): MaxPool2d(kernel_size=15, stride=15, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=40, out_features=1, bias=True)
)

In [26]:
day=True #0 for day, 1 for night
loss_module = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

def train_model(model, optimizer, train_loader, loss_module, num_epochs=3):
    # Set model to train mode
    model.train()

    for epoch in range(num_epochs):
        for data_inputs, data_labels in train_loader:
            for i in range(len(data_labels)):
                if data_labels[i]=='day':
                    data_labels[i]=0
                else:
                    data_labels[i]=1    #day label = 0, night label = 1

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels=torch.FloatTensor(data_labels)
            data_labels = data_labels.to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)
            #print(loss.item(), preds, data_labels)
            
            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()


In [27]:
train_model(model, optimizer, train_loader, loss_module)

KeyboardInterrupt: 

In [ ]:
state_dict = model.state_dict()
torch.save(state_dict, "Classifier_weights.pth")

FileNotFoundError: [Errno 2] No such file or directory: 'Classifier_weights80/20.pth'

In [ ]:
def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs, data_labels in data_loader:
            for i in range(len(data_labels)):
                if data_labels[i]=='day':
                    data_labels[i]=0
                else:
                    data_labels[i]=1    #day label = 0, night label = 1
            data_labels=torch.FloatTensor(data_labels)

            
            # Determine prediction of model on dev set
            data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)
            preds = torch.sigmoid(preds) # Sigmoid to map predictions between 0 and 1
            pred_labels = torch.round(preds) # round predictions to 0 and 1
            pred_domain='day' if pred_labels==0 else 'night'
            true_domain='day' if data_labels==0 else 'night'
            #print(f'Prediction: {pred_domain}   \t    Truth: {true_domain}')
            #print(pred_labels, data_labels)

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (pred_labels == data_labels).sum()
            num_preds += data_labels.shape[0]

    acc = true_preds / num_preds
    print(f"Accuracy of the model: {100.0*acc:4.2f}%")

In [28]:
state_dict = torch.load("Classifier_weights.pth")
same_model = dnClassifier(img_width, img_height)
same_model.load_state_dict(state_dict)
same_model.to(device)
eval_model(model, test_loader)

KeyboardInterrupt: 